# temp
# temp
# temp
# temp

refer to:
- [超实用的7种 pytorch 网络可视化方法](https://blog.csdn.net/Wenyuanbo/article/details/118937790)
- https://bhosmer.github.io/mm/ref.html todo
- [如何可视化 PyTorch 模型](https://dl.ypw.io/visualize-pytorch-model/)

工具 | 是否可用| 更新频率 | 优点 | 缺点
:- | :- | :-  | :-  | :- 
Netron | 可用 | 高 | 在线可视化，不用安装 | 需要保存为 onnx 格式，看不到输入输出的尺寸
tensorboard | 可用 | 高 | 可交互式展开，可视化效果好 | 需要安装 tensorboard，并且启动后台服务
torchview | 可用 | 1 年 | 可以看到每一层的输入输出尺寸 | 需要安装 torchview 和 graphviz
torchviz | 可用 | 3 年 | 无 | 看到的是反向传播的路径，不是模型结构
print 大法 | 永久可用 | 无 | 永久可用，不会失效 | 只有文字，无法展示跨层连接
tensorwatch | 失效 | 4 年 | 无 | 无
hiddenlayer | 失效 | 4 年 | 无 | 无

# [torchview](https://github.com/mert-kurttutan/torchview)
- pip install graphviz
- apt-get install graphviz

# [TensorWatch](https://github.com/microsoft/tensorwatch)
- pip install tensorwatch
- [pytorch 网络可视化(三): Jupyter Notebook + tensorwatch](https://blog.csdn.net/Wenyuanbo/article/details/118579431)


# [tensorboardX](https://github.com/lanpa/tensorboardX)
- pip install tensorboardX
- [pytorch 网络可视化(四)：tensorboardX](https://blog.csdn.net/Wenyuanbo/article/details/118605418)


# [Netron](https://github.com/lutzroeder/Netron)
- pip install netron
- [pytorch 网络可视化(五): netron](https://blog.csdn.net/Wenyuanbo/article/details/118736258)
- [pytorch可视化工具netron](https://www.cnblogs.com/jiangnanyanyuchen/p/13344217.html)
- [PyTorch 神经网络模型可视化(Netron)](https://developer.aliyun.com/article/1300462)

# [PyTorchViz](https://github.com/szagoruyko/pytorchviz)
- apt-get install graphviz
- pip install torchviz

## [HiddenLayer](https://github.com/waleedka/hiddenlayer)
- conda install graphviz python-graphviz
- pip install hiddenlayer
- [pytorch 网络可视化(六): hiddenlayer](https://blog.csdn.net/Wenyuanbo/article/details/118765044)

# print
## [torchsummaryX](https://github.com/nmhkahn/torchsummaryX)
- pip install torchsummaryX

### [pytorch-summary](https://github.com/sksq96/pytorch-summary)
- pip install torchsummary
- [pytorch 网络可视化(一): torchsummary](https://blog.csdn.net/Wenyuanbo/article/details/118514709)

## [torchinfo](https://github.com/TylerYep/torchinfo)
- conda install -c conda-forge torchinfo